In [ ]:
!pip install pandas scikit-learn tensorflow


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN
from sklearn.neural_network import MLPClassifier

In [ ]:
train_df = pd.read_csv('labelled_train_set (1).csv')
test_df = pd.read_csv('unlabelled_test2 (1).csv')
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

NameError: name 'y_train' is not defined

In [ ]:
label_mapping = {'Human-written': 0, 'AI-generated': 1}
train_df['Type'] = train_df['Type'].map(label_mapping)

In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    words = word_tokenize(text)
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)


In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
train_df['Article'] = train_df['Article'].apply(preprocess_text)
test_df['Article'] = test_df['Article'].apply(preprocess_text)


In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
X = train_df['Article']
y = train_df['Type']

In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(test_df['Article'])

In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of y_train:", y_train.shape)
print("Number of samples in y_train before SMOTE:", len(y_train))

Shape of X_train_tfidf: (400, 5000)
Shape of y_train: (400,)
Number of samples in y_train before SMOTE: 400


In [ ]:
print(y_train.isnull().sum())

nan_rows = y_train[y_train.isnull()]
print(nan_rows)

0
Series([], Name: Type, dtype: int64)


In [ ]:
# non_nan_indices = y_train.dropna().index
# X_train_tfidf = X_train_tfidf[non_nan_indices]
# y_train = y_train.dropna()

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_smote , y_train_smote)
rf_val_predictions = rf_model.predict(X_val_tfidf)

In [ ]:
rf_accuracy = accuracy_score(y_val, rf_val_predictions)
print(f"Random Forest Validation Accuracy: {rf_accuracy}")
print("\nRandom Forest Classification Report:")
print(classification_report(y_val, rf_val_predictions))

Random Forest Validation Accuracy: 0.89

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        77
           1       1.00      0.52      0.69        23

    accuracy                           0.89       100
   macro avg       0.94      0.76      0.81       100
weighted avg       0.90      0.89      0.88       100



In [ ]:
model = LogisticRegression()
model.fit(X_train_smote, y_train_smote)

LogisticRegression()

In [ ]:
val_predictions = model.predict(X_val_tfidf)
accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_val, val_predictions))

Validation Accuracy: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91        77
           1       0.75      0.52      0.62        23

    accuracy                           0.85       100
   macro avg       0.81      0.73      0.76       100
weighted avg       0.84      0.85      0.84       100



In [ ]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_smote, y_train_smote)

SVC(kernel='linear', random_state=42)

In [ ]:
svm_val_predictions = svm_model.predict(X_val_tfidf)
svm_accuracy = accuracy_score(y_val, svm_val_predictions)

print(f"SVM Validation Accuracy: {svm_accuracy}")


print("\nSVM Classification Report:")
print(classification_report(y_val, svm_val_predictions))


SVM Validation Accuracy: 0.82

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        81
           1       0.53      0.53      0.53        19

    accuracy                           0.82       100
   macro avg       0.71      0.71      0.71       100
weighted avg       0.82      0.82      0.82       100



In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_smote, y_train_smote)
xgb_val_predictions = xgb_model.predict(X_val_tfidf)

xgb_accuracy = accuracy_score(y_val, xgb_val_predictions)
print(f"XGBoost Validation Accuracy: {xgb_accuracy}")

print("\nXGBoost Classification Report:")
print(classification_report(y_val, xgb_val_predictions))


XGBoost Validation Accuracy: 0.82

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        81
           1       0.53      0.53      0.53        19

    accuracy                           0.82       100
   macro avg       0.71      0.71      0.71       100
weighted avg       0.82      0.82      0.82       100



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_smote.toarray(), y_train_smote, epochs=10, batch_size=32, validation_data=(X_val_tfidf.toarray(), y_val))

nn_val_predictions = (model.predict(X_val_tfidf.toarray()) > 0.5).astype("int32")

nn_accuracy = accuracy_score(y_val, nn_val_predictions)
print(f"Neural Network Validation Accuracy: {nn_accuracy}")

print("\nNeural Network Classification Report:")
print(classification_report(y_val, nn_val_predictions))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.5559 - loss: 0.6894 - val_accuracy: 0.2300 - val_loss: 0.7058
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7241 - loss: 0.6487 - val_accuracy: 0.7000 - val_loss: 0.6621
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9357 - loss: 0.5446 - val_accuracy: 0.8500 - val_loss: 0.5279
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9743 - loss: 0.3301 - val_accuracy: 0.8400 - val_loss: 0.4184
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9847 - loss: 0.1383 - val_accuracy: 0.8300 - val_loss: 0.3866
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9905 - loss: 0.0651 - val_accuracy: 0.8200 - val_loss: 0.4205
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9993 - loss: 0.0264 - val_accuracy: 0.8200 - val_loss: 0.4537
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0139 - val_accuracy: 0.8100 - v

In [ ]:
from sklearn.ensemble import VotingClassifier
lr_model = LogisticRegression()
voting_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('svm', svm_model),
    ('lr', lr_model),

], voting='hard')

voting_model.fit(X_train_smote, y_train_smote)
voting_val_predictions = voting_model.predict(X_val_tfidf)

voting_accuracy = accuracy_score(y_val, voting_val_predictions)
print(f"Voting Classifier Validation Accuracy: {voting_accuracy}")

print("\nVoting Classifier Classification Report:")
print(classification_report(y_val, voting_val_predictions))


Voting Classifier Validation Accuracy: 0.86

Voting Classifier Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.98      0.92        81
           1       0.78      0.37      0.50        19

    accuracy                           0.86       100
   macro avg       0.82      0.67      0.71       100
weighted avg       0.85      0.86      0.84       100



In [ ]:
param_grid = {
    'n_estimators': [90 ,100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 9 , 10],
    'min_samples_leaf': [1, 2, 4 , 9 , 10],
    'bootstrap': [True, False]
}

In [ ]:
rf_model = RandomForestClassifier(random_state=42)


In [ ]:
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)


In [ ]:
grid_search.fit(X_train_smote, y_train_smote)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [ ]:
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

In [ ]:
# print(f"Best Parameters for Random Forest: {best_params}")
# rf_val_predictions = best_rf_model.predict(X_val_tfidf)
# rf_accuracy = accuracy_score(y_val, rf_val_predictions)

Best Parameters for Random Forest: {'bootstrap': False, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


In [ ]:
# print(f"Random Forest Validation Accuracy with Hyperparameter Tuning: {rf_accuracy}")
# print("\nRandom Forest Classification Report with Hyperparameter Tuning:")
# print(classification_report(y_val, rf_val_predictions))

Random Forest Validation Accuracy with Hyperparameter Tuning: 0.87

Random Forest Classification Report with Hyperparameter Tuning:
              precision    recall  f1-score   support

           0       0.88      0.98      0.92        81
           1       0.80      0.42      0.55        19

    accuracy                           0.87       100
   macro avg       0.84      0.70      0.74       100
weighted avg       0.86      0.87      0.85       100



In [ ]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100 , 110 , 150],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}


In [ ]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)


In [ ]:
grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available cores
    verbose=2  # Print detailed progress
)


In [ ]:
grid_search.fit(X_train_smote, y_train_smote)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
225 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklear

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=2)

In [ ]:
best_params = grid_search.best_params_
best_log_reg = grid_search.best_estimator_

print(f"Best Parameters for Logistic Regression: {best_params}")


Best Parameters for Logistic Regression: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}


In [ ]:
val_predictions = best_log_reg.predict(X_val_tfidf)
accuracy = accuracy_score(y_val, val_predictions)

print(f"Logistic Regression Validation Accuracy with Hyperparameter Tuning: {accuracy}")

print("\nLogistic Regression Classification Report with Hyperparameter Tuning:")
print(classification_report(y_val, val_predictions))


Logistic Regression Validation Accuracy with Hyperparameter Tuning: 0.85

Logistic Regression Classification Report with Hyperparameter Tuning:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91        77
           1       0.75      0.52      0.62        23

    accuracy                           0.85       100
   macro avg       0.81      0.73      0.76       100
weighted avg       0.84      0.85      0.84       100

